<a href="https://colab.research.google.com/github/masaya-T/Signate-ws/blob/main/Signate_%E7%B7%B4%E7%BF%92%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備
## pip install

In [ ]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

     |████████████████████████████████| 138 kB 9.5 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=bfbc59d44d860bf45d3a6250c84259bacd6029a4a802c605688a4d714f8401d2
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Download 100%.


In [ ]:
! signate list

/bin/bash: signate: command not found


# コンペデータ

## 【練習問題】自動車の走行距離予測

自動車の属性データからガソリン１ガロンあたりの走行距離を予測するモデルを作成していただきます

自動車が誕生したのは1769年。私達の生活を劇的に変化させました。現代も、電気自動車や自動運転技術など進化を続けています。

ここでは、自動車の属性データから走行距離の予測にチャレンジしていただきます。

本コンペを活用して、SIGNATEでのデータ解析・モデル構築を体験してください。

データ概要

課題種別：回帰
データ種別：多変量
学習データサンプル数：199
説明変数の数：8
欠損値：あり

|カラム	|ヘッダ名称	|データ型	|説明|
|-|-|-|-|
|0|	id	|int	|インデックスとして使用|
|1|	mpg	|float	|ガソリン１ガロンあたりの走行距離|（mile par gallon の略）
|2|	cylinders|	varchar|	シリンダー
|3|	displacement|	float|	排気量
|4|	horsepower|	float|	馬力
|5|	weight|	float	|重量
|6|	acceleration	|float|	加速度
|7|	model |year	varchar	|年式
|8|	origin|	varchar|	起源
|9|	car name|	varchar|	車名

In [ ]:
! signate download --competition-id=121

sample_submit.csv

test.tsv

train.tsv


Download completed.


## 実装
### データ用意

In [35]:
from re import X
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

train_data=pd.read_table('/content/drive/MyDrive/自動車の走行距離予測/train.tsv')
test_data=pd.read_table('/content/drive/MyDrive/自動車の走行距離予測/test.tsv')

#cols=['id', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']
cols=['displacement', 'horsepower', 'weight']

# テストデータをランダムサンプリングで検証データを抽出
valid_data=train_data.sample(frac=0.1, random_state=0)
# 元データからdf_testのインデックスに該当するデータを除外する
train_data = train_data.drop(valid_data.index)

train_data=train_data[train_data['horsepower'] != '?']
valid_data=valid_data[valid_data['horsepower'] != '?']
test_data=test_data[test_data['horsepower'] != '?']

y_train=train_data['mpg'].values
x_train=train_data[cols]
x_train['horsepower']=train_data['horsepower'].astype(float)

y_valid=valid_data['mpg'].values
x_valid=valid_data[cols]
x_valid['horsepower']=x_valid['horsepower'].astype(float)

x_test=test_data[cols]

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_train.dtypes)


(176, 3) (176,)
(19, 3) (19,)
displacement    float64
horsepower      float64
weight          float64
dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## データ確認


In [21]:
from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=6) 
selector.fit(x_train,y_train)
mask = selector.get_support()    # 各特徴量を選択したか否かのmaskを取得

print(x_train.columns.tolist())
print(mask)
print(x_train.columns[mask].tolist())

['id', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']
[False  True  True  True  True False  True  True]
['cylinders', 'displacement', 'horsepower', 'weight', 'model year', 'origin']


In [22]:
from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=3) 
selector.fit(x_train,y_train)
mask = selector.get_support()    # 各特徴量を選択したか否かのmaskを取得

print(x_train.columns.tolist())
print(mask)
print(x_train.columns[mask].tolist())

['id', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']
[False False  True  True  True False False False]
['displacement', 'horsepower', 'weight']


# 線形回帰

In [13]:
from sklearn.linear_model import LinearRegression


def predictLogisticRegression(X_train,Y_train,X_test,Y_test):
    # モデル作成（ロジスティック回帰）
    # model = LinearRegression(normalize=True)
    model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())
    # 学習
    model.fit(X_train, Y_train)
    # 予測
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    # 学習データを用いたときの平均二乗誤差を出力
    train_loss=mean_squared_error(Y_train, y_train_pred)
    # 検証データを用いたときの平均二乗誤差を出力
    valid_loss=mean_squared_error(Y_test, y_pred)
    print('MSE train data: ', train_loss) 
    print('MSE test data: ', valid_loss)         
    
    return 0

predictLogisticRegression(x_train,y_train,x_valid,y_valid)

MSE train data:  18.56199609935722
MSE test data:  12.077008693503076


0

# ElasticNet回帰

In [14]:
from sklearn.linear_model import ElasticNet

def predictElasticNet(X_train,Y_train,X_test,Y_test):
  # モデル作成（ElasticNet回帰）
  model = ElasticNet(alpha=0.05)
  model.fit(X_train, Y_train)

  # 回帰　
  y_train_pred = model.predict(X_train)
  y_pred= model.predict(X_test)
  # 評価
  # 決定係数(R2)
  r2_elasticnet = r2_score(Y_train, y_train_pred)
  r2_elasticnet = r2_score(Y_test, y_pred)

  # 学習データを用いたときの平均二乗誤差を出力
  train_loss=mean_squared_error(Y_train, y_train_pred)
  # 検証データを用いたときの平均二乗誤差を出力
  valid_loss=mean_squared_error(Y_test, y_pred)
  
  print('MSE train data: ', train_loss) 
  print('MSE test data: ', valid_loss)        

  # 回帰係数
  print("Coef = ", model.coef_)
  
predictElasticNet(x_train,y_train,x_valid,y_valid)

MSE train data:  18.5619983888248
MSE test data:  12.07981460185344
Coef =  [-0.01002656 -0.05365063 -0.00460504]


# RandomForest回帰

In [41]:
from sklearn.ensemble import RandomForestRegressor

def predictRandomForest(X_train,Y_train,X_test,Y_test):
  # モデル作成（RandomForest回帰）
  model = RandomForestRegressor()
  model.fit(X_train, Y_train)

  # 回帰　
  y_train_pred = model.predict(X_train)
  y_pred= model.predict(X_test)
  # 評価

  # 学習データを用いたときの平均二乗誤差を出力
  train_loss=mean_squared_error(Y_train, y_train_pred)
  # 検証データを用いたときの平均二乗誤差を出力
  valid_loss=mean_squared_error(Y_test, y_pred)
  
  print('MSE train data: ', train_loss) 
  print('MSE test data: ', valid_loss)    

  # 変数重要度
  print("feature_importances = ", model.feature_importances_)  
predictRandomForest(x_train,y_train,x_valid,y_valid)

MSE train data:  2.517595999999998
MSE test data:  14.509540578947382
feature_importances =  [0.44656733 0.31738884 0.23604383]


# GBDT（勾配ブースティング木）

In [38]:
from sklearn.ensemble import GradientBoostingRegressor

def predictGBDT(X_train,Y_train,X_test,Y_test):
  # モデル作成（勾配ブースティング木）
  model = GradientBoostingRegressor()
  model.fit(X_train, Y_train)

  # 回帰　
  y_train_pred = model.predict(X_train)
  y_pred= model.predict(X_test)
  # 評価

  # 学習データを用いたときの平均二乗誤差を出力
  train_loss=mean_squared_error(Y_train, y_train_pred)
  # 検証データを用いたときの平均二乗誤差を出力
  valid_loss=mean_squared_error(Y_test, y_pred)
  
  print('MSE train data: ', train_loss) 
  print('MSE test data: ', valid_loss)    

  # 変数重要度
  print("feature_importances = ", model.feature_importances_)  
predictGBDT(x_train,y_train,x_valid,y_valid)

MSE train data:  2.6262446251345377
MSE test data:  15.804602920342553
feature_importances =  [0.56742039 0.2731126  0.15946701]


#  SVR（サポートベクターマシーン）

In [37]:
from sklearn.svm import SVR

def predictSVR(X_train,Y_train,X_test,Y_test):
  # モデル作成（RandomForest回帰）
  model = SVR(kernel='linear', C=1, epsilon=0.1, gamma='auto')
  model.fit(X_train, Y_train)

  # 回帰　
  y_train_pred = model.predict(X_train)
  y_pred= model.predict(X_test)
  # 評価

  # 学習データを用いたときの平均二乗誤差を出力
  train_loss=mean_squared_error(Y_train, y_train_pred)
  # 検証データを用いたときの平均二乗誤差を出力
  valid_loss=mean_squared_error(Y_test, y_pred)
  
  print('MSE train data: ', train_loss) 
  print('MSE test data: ', valid_loss)    

  # 変数重要度
  print("feature_importances = ", model.feature_importances_)  
predictRandomForest(x_train,y_train,x_valid,y_valid)

MSE train data:  2.532188267045452
MSE test data:  12.148343684210541
feature_importances =  [0.47098805 0.26397951 0.26503243]


# XGBoost

In [43]:
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

def predictXGBoostCV(X_train,Y_train,X_test,Y_test):
    # xgboostモデルの作成
    model = xgb.XGBRegressor()
    model.fit(X_train, Y_train)

    # 回帰　
    y_train_pred = model.predict(X_train)
    y_pred= model.predict(X_test)
    # 評価

    # 学習データを用いたときの平均二乗誤差を出力
    train_loss=mean_squared_error(Y_train, y_train_pred)
    # 検証データを用いたときの平均二乗誤差を出力
    valid_loss=mean_squared_error(Y_test, y_pred)
    
    print('MSE train data: ', train_loss) 
    print('MSE test data: ', valid_loss)    

    # 変数重要度
    print("feature_importances = ", model.feature_importances_)  

predictXGBoostCV(x_train,y_train,x_valid,y_valid)

[13:21:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE train data:  4.109376300918567
MSE test data:  15.731012095404962
feature_importances =  [0.53060484 0.34756213 0.12183305]


# Grid Seatch

In [45]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
def GridSearch(X_train,Y_train,X_test,Y_test):
    
    '''-------------------------------------------------------------------------------------------------------'''
    # xgboostモデルの作成
    model_xgb = xgb.XGBRegressor()
    model_xgb.fit(X_train, Y_train)
     # ハイパーパラメータ探索
    clf_cv = GridSearchCV(model_xgb, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)
    clf_cv.fit(X_train, Y_train)

    # 回帰　
    y_train_pred = clf_cv.predict(X_train)
    y_pred= clf_cv.predict(X_test)
    # 学習データを用いたときの平均二乗誤差を出力
    train_loss=mean_squared_error(Y_train, y_train_pred)
    # 検証データを用いたときの平均二乗誤差を出力
    valid_loss=mean_squared_error(Y_test, y_pred)
    
    print('MSE train data: ', train_loss) 
    print('MSE test data: ', valid_loss)    

    '''-------------------------------------------------------------------------------------------------------'''
    # モデル作成（勾配ブースティング木）
    model_GBR = GradientBoostingRegressor()
    model_GBR.fit(X_train, Y_train)
    # ハイパーパラメータ探索
    clf_cv = GridSearchCV(model_GBR, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)
    clf_cv.fit(X_train, Y_train)

    # 回帰　
    y_train_pred = clf_cv.predict(X_train)
    y_pred= clf_cv.predict(X_test)
    # 学習データを用いたときの平均二乗誤差を出力
    train_loss=mean_squared_error(Y_train, y_train_pred)
    # 検証データを用いたときの平均二乗誤差を出力
    valid_loss=mean_squared_error(Y_test, y_pred)
    
    print('MSE train data: ', train_loss) 
    print('MSE test data: ', valid_loss)

    '''-------------------------------------------------------------------------------------------------------'''
    # モデル作成（RandomForest回帰）
    model_RFR = RandomForestRegressor()
    model_RFR.fit(X_train, Y_train)
    # ハイパーパラメータ探索
    clf_cv = GridSearchCV(model_RFR, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)
    clf_cv.fit(X_train, Y_train)

    # 回帰　
    y_train_pred = clf_cv.predict(X_train)
    y_pred= clf_cv.predict(X_test)
    # 学習データを用いたときの平均二乗誤差を出力
    train_loss=mean_squared_error(Y_train, y_train_pred)
    # 検証データを用いたときの平均二乗誤差を出力
    valid_loss=mean_squared_error(Y_test, y_pred)
    
    print('MSE train data: ', train_loss) 
    print('MSE test data: ', valid_loss)   
GridSearch(x_train,y_train,x_valid,y_valid)


[13:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[13:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:28:00] WARNING: /workspace/src/objective/regres